# **Loading Data**

## **Loading Data into Calendar table**

In [9]:
USE TargetMartII


IF (SELECT count(*) FROM dim.Calendar) = 0
BEGIN
-- Declare variables
DECLARE @StartDate DATE = '2020-01-01'
DECLARE @EndDate DATE = DATEADD(year, 0, GETDATE())
DECLARE @Date DATE = @StartDate
DECLARE @DayID INT = (datepart(year, @StartDate)-1900)*1000 + datepart(dy, @StartDate)
;

--This is where we fill the data in
WHILE @Date <= @EndDate
	BEGIN
		INSERT INTO dim.Calendar (pkCalendar, DateValue, Year, Quarter, Qtr, Month,  MonthName, MonthShort, Week, Day, DayName, DayShort, IsWeekday, Weekday)
		VALUES (
			@DayID,
			@Date,
			YEAR(@Date),
			DATEPART(QUARTER, @Date),
			CASE WHEN DATEPART(QUARTER, @Date) IN (1) THEN '1st'
				 WHEN DATEPART(QUARTER, @Date) IN (2) THEN '2nd'
				 WHEN DATEPART(QUARTER, @Date) IN (3) THEN '3rd'
				 WHEN DATEPART(QUARTER, @Date) IN (4) THEN '4th'
				 ELSE '5th'
				 END,
			MONTH(@Date),
			DATENAME(MONTH, @Date),
			LEFT(DATENAME(MONTH, @Date),3),
			DATEPART(WEEK, @Date),
			DAY(@Date),
			DATENAME(WEEKDAY, @Date),
			LEFT(DATENAME(WEEKDAY, @Date),3),
			CASE WHEN DATEPART(WEEKDAY, @Date) IN (1, 7) THEN 0 ELSE 1 END, -- Set IsWeekday to 0 for Saturday (1) and Sunday (7), and 1 for weekdays
			CASE WHEN DATEPART(WEEKDAY, @Date) IN (1, 7) THEN 'No' ELSE 'Yes' END
	 )

		-- Increment the date and day ID
		SET @Date = DATEADD(DAY, 1, @Date)
		SET @DayID = @DayID + 1
	END
END
GO

Commands completed successfully.

Total execution time: 00:00:00.015

In [11]:
Select TOP 5 *
FROM TargetMartII.dim.Calendar
;

(5 rows affected)

Total execution time: 00:00:00.034

pkCalendar,DateValue,Year,Quarter,Qtr,Month,MonthName,MonthShort,Week,Day,DayName,DayShort,IsWeekday,Weekday
120001,2020-01-01,2020,1,1st,1,January,Jan,1,1,Wednesday,Wed,1,Yes
120002,2020-01-02,2020,1,1st,1,January,Jan,1,2,Thursday,Thu,1,Yes
120003,2020-01-03,2020,1,1st,1,January,Jan,1,3,Friday,Fri,1,Yes
120004,2020-01-04,2020,1,1st,1,January,Jan,1,4,Saturday,Sat,0,No
120005,2020-01-05,2020,1,1st,1,January,Jan,2,5,Sunday,Sun,0,No


The above script checks if the dim.Calendar table is empty and, if so, loads it with date-related information within a specified range. It inserts records that includes information such as the year, quarter, month, and weekday indications using a loop. The script guarantees that weekends are properly detected. The procedure is complete after the table is populated with comprehensive date-related data. The top 5 rows from the dim.Calendar table are shown in the second cell.

## **Loading Data into dim.Product table**

In [12]:
USE TargetMartII

INSERT INTO dim.Products(pkProdId, ProductId, Product, ProductCategory, CategoryDesc, UnitPrice, UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued, ReorderFlag)
SELECT prod.ProductID + 10000 as 'pkProdId'
      , prod.ProductID as 'ProductId'
      , prod.ProductName as 'Product'
	  , cat.CategoryName as 'ProductCategory'
	  , cat.[Description] as 'CategoryDesc'
	  , prod.UnitPrice
	  , prod.UnitsInStock
	  , prod.UnitsOnOrder
	  , prod.ReorderLevel
	  , prod.Discontinued
	  , CASE 
			WHEN prod.ReorderLevel >= (prod.UnitsInStock + prod.UnitsOnOrder)
			     AND prod.Discontinued = 0
			 THEN 'Yes'
	      ELSE 'No'
	    END as 'ReorderFlag'
FROM Northwind_2023.dbo.Products prod
	INNER JOIN Northwind_2023.dbo.Categories cat
	ON prod.CategoryID = cat.CategoryID
WHERE prod.ProductID not in (SELECT ProductId FROM dim.Products)
;

GO

(0 rows affected)

Total execution time: 00:00:00.044

In [13]:
SELECT TOP 5 *
FROM TargetMartII.dim.Products
;

(5 rows affected)

Total execution time: 00:00:00.050

pkProdId,ProductId,Product,ProductCategory,CategoryDesc,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,ReorderFlag
10001,1,Chai,Beverages,"Soft drinks, coffees, teas, beers, and ales",18.00,39,0,10,0,No
10002,2,Chang,Beverages,"Soft drinks, coffees, teas, beers, and ales",19.00,17,40,25,0,No
10003,3,Aniseed Syrup,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",10.00,13,70,25,0,No
10004,4,Chef Anton's Cajun Seasoning,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",22.00,53,0,0,0,No
10005,5,Chef Anton's Gumbo Mix,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",21.35,0,0,0,1,No


The above script extracts and transforms data from Northwind\_2023.dbo.Products and inserts it into the dim.Products table. It modifies the 'pkProdId' by adding 10,000 to the ProductID and assigns a 'ReorderFlag' depending on certain conditions. Only new records are added by filtering out existing ProductID values in the target table. The second column simply shows that the data extraction and processing is complete, and it displays the top 5 records of dim.products.

## **Loading Data into dim.Employee table**

In [14]:
use TargetMartII

INSERT INTO dim.Employees(EmployeeID, Employee, Title, BirthDate, HireDate, City, Country, ReportsTo)
SELECT sEmp.EmployeeID
	  ,concat( sEmp.TitleOfCourtesy, ' ', sEmp.FirstName, ', ', sEmp.LastName)
      ,sEmp.Title
      ,sEmp.BirthDate
      ,sEmp.HireDate
      ,sEmp.City
      ,sEmp.Country
      ,sEmp.ReportsTo
FROM Northwind_2023.dbo.Employees sEmp
WHERE sEmp.EmployeeID not in (SELECT EmployeeID FROM dim.Employees)

(0 rows affected)

Total execution time: 00:00:00.025

In [15]:
SELECT TOP 5 *
FROM TargetMartII.dim.Employees
;

(5 rows affected)

Total execution time: 00:00:00.021

EmployeeID,Employee,Title,BirthDate,HireDate,City,Country,ReportsTo
1,"Ms. Nancy, Davolio",Sales Representative,1973-12-08 00:00:00.000,2017-05-01 00:00:00.000,Seattle,USA,2
2,"Dr. Andrew, Fuller","Vice President, Sales",1977-02-19 00:00:00.000,2017-08-14 00:00:00.000,Tacoma,USA,NULL
3,"Ms. Janet, Leverling",Sales Representative,1988-08-30 00:00:00.000,2017-04-01 00:00:00.000,Kirkland,USA,2
4,"Mrs. Margaret, Peacock",Sales Representative,1962-09-19 00:00:00.000,2018-05-03 00:00:00.000,Redmond,USA,2
5,"Mr. Steven, Buchanan",Sales Manager,1980-03-04 00:00:00.000,2018-10-17 00:00:00.000,London,UK,2


The above script inserts expanded employee data into the dim.Employees table. It extracts and changes data from Northwind\_2023.dbo.Employees, create the 'Employee' column by concatenating an employee's title, first name, last name, and other details. The script inserts new records carefully, excluding those with existing EmployeeID values in the target table.

## **Loading Data into dim.Shipper table**

In [16]:
use TargetMartII

INSERT INTO dim.Shipper(pkShipId, Shipper, Phone)
SELECT s_sh.ShipperID
      ,s_sh.CompanyName
      ,s_sh.Phone
FROM Northwind_2023.dbo.Shippers s_sh
WHERE s_sh.ShipperID not in (SELECT pkShipId FROM dim.Shipper)
;
GO

(0 rows affected)

Total execution time: 00:00:00.013

In [17]:
SELECT *
FROM TargetMartII.dim.Shipper
;

(3 rows affected)

Total execution time: 00:00:00.009

pkShipId,Shipper,Phone
1,Speedy Express,(503) 555-9831
2,United Package,(503) 555-3199
3,Federal Shipping,(503) 555-9931


The source table in this script is dbo.Shippers, and the database is Northwind\_2023. Its data is being created and imported into the new dim.Shipper.

## **Loading Data into dim.Customer table**

In [18]:
use TargetMartII

INSERT INTO dim.Customers(CustomerID, Customer, City, Country, LoadDate, SourceCountry)
	SELECT nwc.CustomerID
		  ,nwc.CompanyName 
		  ,nwc.City
		  ,nwc.Country
		  ,CAST(getdate() as DATE)
		  ,concat('Country = ', nwc.Country) 
	FROM Northwind_2023.dbo.Customers nwc
	WHERE nwc.CustomerID not in (SELECT CustomerID FROM dim.Customers)

;
GO

(0 rows affected)

Total execution time: 00:00:00.017

In [19]:
SELECT TOP 5 *
FROM dim.Customers
;

(5 rows affected)

Total execution time: 00:00:00.017

pkCustomer,CustomerID,Customer,City,Country,LoadDate,SourceCountry
1000,ALFKI,Alfreds Futterkiste,Berlin,Germany,2023-11-27,Country = Germany
1001,ANATR,Ana Trujillo Emparedados y helados,México D.F.,Mexico,2023-11-27,Country = Mexico
1002,ANTON,Antonio Moreno Taquería,México D.F.,Mexico,2023-11-27,Country = Mexico
1003,AROUT,Around the Horn,London,UK,2023-11-27,Country = UK
1004,BERGS,Berglunds snabbköp,Luleå,Sweden,2023-11-27,Country = Sweden


The source table in this script is dbo.Northwind\_2023 database. Its data is being obtained and imported into the new dim.Customers.

## **Loading Data into f.OrderPerf table**

In [22]:
TRUNCATE TABLE TargetMartII.f.OrderPerf
GO

INSERT INTO TargetMartII.f.OrderPerf(OrderID, fkCalendar, fkCustomer, fkShipper, fkEmployee, Freight, DaysToShip, DaysTilRequired, OrderCount)
SELECT sO.OrderID
	  ,tC.pkCalendar as fkCalendar
	  ,tCus.pkCustomer as fkCustomer
	  ,sO.ShipVia as fkShipper
	  ,sO.EmployeeID as fkEmployee
	  ,sO.Freight
	  ,DATEDIFF ( day , sO.OrderDate , sO.ShippedDate ) as 'DaysToShip'
	  ,DATEDIFF ( day , sO.OrderDate , sO.RequireDate ) as 'DaysTilRequired'
	  , 1 as 'OrderCount'
FROM Northwind_2023.dbo.Orders sO
	INNER JOIN TargetMartII.dim.Calendar tC
	ON cast(so.OrderDate as DATE) = tc.DateValue
	INNER JOIN TargetMartII.dim.Customers tCus
	ON sO.CustomerID = tCus.CustomerID
; 


Commands completed successfully.

(830 rows affected)

Total execution time: 00:00:00.025

In [23]:
SELECt TOP 5 *
FROM TargetMartII.f.OrderPerf
;

(5 rows affected)

Total execution time: 00:00:00.034

OrderID,fkCalendar,fkCustomer,fkShipper,fkEmployee,Freight,DaysToShip,DaysTilRequired,OrderCount
10248,120551,1084,3,5,32.38,12,28,1
10249,120552,1078,1,6,11.61,5,42,1
10250,120555,1033,2,4,65.83,4,28,1
10251,120555,1083,1,3,41.34,7,28,1
10252,120556,1075,2,4,51.30,2,28,1


The data from Northwnd\_2023 is being converted and transferred here from multiple tables and columns. For instance, dbo.Orders placed by Northwind\_2023, dim.Customers and Calendar from TargetMart2. The Truncate table command is used to avoid having to update the table every time we insert new data. Simply truncating the table allows us to erase the data and insert new and updated data inside the same operation.

## **Loading Data into f.ProductPerf table**

In [24]:
TRUNCATE TABLE TargetMartII.f.ProductPerf
GO

INSERT INTO TargetMartII.f.ProductPerf(OrderID, fkOrderDate, fkProductID, fkCustomerID, UnitPrice, Quantity, Discount, DiscFlag, LineTotal)
SELECT sDet.OrderID
      ,tCal.pkCalendar as 'OrderDate'
      ,sDet.ProductID + 10000 as 'ProductID'
--	  ,sOrd.CustomerID
	  ,tCust.pkCustomer
      ,sDet.UnitPrice
      ,sDet.Quantity
      ,sDet.Discount
	  ,CASE 
		 WHEN sDet.Discount = 0 THEN 0
		 ELSE 1
	   END as 'DiscFlag'
	  ,CAST(((1 - sDet.Discount) * sDet.UnitPrice) * sDet.Quantity as MONEY) as 'LineTotal'
FROM Northwind_2023.dbo.OrderDetails sDet
	INNER JOIN Northwind_2023.dbo.Orders sOrd
	ON sDet.OrderID = sOrd.OrderID
	INNER JOIN TargetMartII.dim.Calendar tCal
	ON CAST(sOrd.OrderDate as DATE) = tCal.DateValue
	INNER JOIN TargetMartII.dim.Customers tCust
	ON sOrd.CustomerID = tCust.CustomerID
;

Commands completed successfully.

(2155 rows affected)

Total execution time: 00:00:00.087

In [26]:
SELECT TOP 5 *
FROM TargetMartII.f.ProductPerf
;

(5 rows affected)

Total execution time: 00:00:00.007

OrderID,fkOrderDate,fkProductID,fkCustomerID,UnitPrice,Quantity,Discount,DiscFlag,LineTotal
10248,120551,10011,1084,14.00,12,0,0,168.00
10248,120551,10042,1084,9.80,10,0,0,98.00
10248,120551,10072,1084,34.80,5,0,0,174.00
10249,120552,10014,1078,18.60,9,0,0,167.40
10249,120552,10051,1078,42.40,40,0,0,1696.00


The data from Northwnd\_2023 is being converted and transferred here from multiple tables and columns. For instance, dbo.Orders as well as dbo.Northwind\_2023, orderDetails, dim.Customers and Calendar from TargetMart2. The Truncate table command is used to avoid having to update the table every time we insert new data. Simply truncating the table allows us to erase the data and insert new and updated data inside the same operation. The second cell displays the first five rows of the ProductPerf table.